In [1]:
#pip install seaborn
# !pip install openpyxl

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline


In [3]:
# for dirname, _, filenames in os.walk(r'..\data'):
#     dir_name=dirname
dir_name="data"

In [4]:
# dir_name=os.path.join('../','data_1')

# Reading Data files


In [5]:
df_1=pd.read_excel(os.path.join(dir_name,'cl_sell_buy.xlsx'),engine='openpyxl',)
df_2=pd.read_excel(os.path.join(dir_name,'diffs.xlsx'),engine='openpyxl',)  
df_3=pd.read_excel(os.path.join(dir_name,'margins.xlsx'),engine='openpyxl',)

In [6]:
def delete_cols(date_list, del_list):
    for name in df_lag.columns:
            if 't-' in name or name in date_list or name=='target_position':
                pass                        
    
            else:
                del df_lag[name]
            for str_value in del_list:
                if str_value in name:
                    del df_lag[name]
                    

In [7]:
def normalize_df(df):

    # copy the data
    df_min_max_scaled = df.copy()

    # apply normalization techniques
    for column in df_min_max_scaled.columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    

    # view normalized data
    return(df_min_max_scaled)

In [8]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [9]:
start_date = '2015-01-10'

In [10]:
mask = (df_1['date'] > start_date)
df_1 = df_1.loc[mask]
df_1.dropna(axis=0, inplace=True)


In [11]:
df_1.head()

,date,time_spread,dod_change,target_position
1015,2015-01-12,-0.69,-0.06,SHORT
1016,2015-01-13,-0.62,0.07,LONG
1017,2015-01-14,-0.48,0.14,LONG
1018,2015-01-15,-0.48,0.00,LONG
1019,2015-01-16,-0.44,0.04,LONG


In [12]:
df_2.head()

,ObservationDate,MEH-WTI,MID,LLS
0,2015-10-01,0.25,-0.20,2.25
1,2015-10-02,-0.40,-0.20,1.80
2,2015-10-05,-0.45,-0.25,1.60
3,2015-10-06,0.10,-0.20,1.65
4,2015-10-07,0.00,-0.05,2.00


In [13]:
mask = (df_2['ObservationDate'] > start_date)
df_2 = df_2.loc[mask]
df_2.dropna(axis=0, inplace=True)

In [14]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384 entries, 0 to 1383
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ObservationDate  1384 non-null   datetime64[ns]
 1   MEH-WTI          1384 non-null   float64       
 2   MID              1384 non-null   float64       
 3   LLS              1384 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 54.1 KB


In [15]:
df_3.head()

,DateCalculated,Contract,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
0,2013-01-22,2013-02-01,NaN,NaN,5.78,9.29
1,2013-04-22,2013-05-01,NaN,NaN,18.92,19.77
2,2013-05-21,2013-06-01,NaN,NaN,15.99,16.87
3,2013-07-22,2013-08-01,NaN,NaN,-0.67,0.96
4,2013-10-21,2013-11-01,NaN,NaN,1.48,4.42


In [16]:
mask = (df_3['DateCalculated'] > start_date)
df_3 = df_3.loc[mask]
df_3.dropna(axis=0, inplace=True)
df_3.describe()

,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
count,715.000000,715.000000,715.000000,715.000000
mean,2.555455,0.997818,3.782629,5.967385
std,1.995024,2.077284,3.588861,3.573026
min,-1.530000,-3.810000,-3.270000,-1.240000
25%,1.085000,-0.530000,1.275000,3.505000
50%,2.350000,0.820000,3.320000,5.710000
75%,3.840000,2.180000,6.380000,8.395000
max,16.540000,14.740000,57.230000,60.230000


In [17]:
df1_df2=pd.merge(
    df_1,
    df_2,
    left_on=['date'],
    right_on=['ObservationDate']
)

In [18]:
combined_df=pd.merge(
    df1_df2,
    df_3,
    left_on=['date'],
    right_on=['DateCalculated']
)


In [19]:
combined_df['year'] = pd.DatetimeIndex(combined_df['date']).year
combined_df['month'] = pd.DatetimeIndex(combined_df['date']).month
combined_df['month_day'] = pd.DatetimeIndex(combined_df['date']).day
combined_df['weekday'] = pd.DatetimeIndex(combined_df['date']).weekday   # Monday is 0 and Sunday is 6

In [20]:
combined_df=combined_df.drop(columns=['date','ObservationDate','DateCalculated', 'Contract'])

In [21]:
type(combined_df.columns)

pandas.core.indexes.base.Index

In [22]:
lags = range(1, 21)  # 5 lags

df_lag=combined_df.assign(**{'{} (t-{})'.format(col, t): combined_df[col].shift(t)
    for t in lags
    for col in combined_df
})

In [23]:
df_lag.head()

,time_spread,dod_change,target_position,MEH-WTI,MID,LLS,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK,...,MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20),year (t-20),month (t-20),month_day (t-20),weekday (t-20)
0,-1.07,0.26,LONG,-0.50,0.55,1.80,0.87,1.76,1.91,0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.91,0.20,LONG,2.00,0.20,2.75,0.54,0.85,1.33,1.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.61,0.09,LONG,1.85,-0.50,2.35,-1.33,0.28,2.73,2.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.00,-0.41,SHORT,0.80,-0.60,1.85,1.28,2.00,3.47,3.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.36,0.23,LONG,1.65,0.00,1.50,1.60,1.74,5.78,5.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#input_df=combined_df.drop(columns=['action','ObservationDate','DateCalculated', 'Contract'])

In [25]:
print (df_lag.columns)

Index(['time_spread', 'dod_change', 'target_position', 'MEH-WTI', 'MID', 'LLS',
       'MEH | USGC | HSK', 'MEH | USGC | FCC', 'WTI | P2 | FCC',
       'WTI | P2 | HSK',
       ...
       'MID (t-20)', 'LLS (t-20)', 'MEH | USGC | HSK (t-20)',
       'MEH | USGC | FCC (t-20)', 'WTI | P2 | FCC (t-20)',
       'WTI | P2 | HSK (t-20)', 'year (t-20)', 'month (t-20)',
       'month_day (t-20)', 'weekday (t-20)'],
      dtype='object', length=294)


In [26]:
dates_list=['year','month','weekday','month_day']
del_str=['year (t', 'month (t', 'weekday (t', 'month_day (t' ]

In [27]:
delete_cols(dates_list,del_str)

In [28]:
print(df_lag.isnull().sum())

target_position             0
year                        0
month                       0
month_day                   0
weekday                     0
                           ..
LLS (t-20)                 20
MEH | USGC | HSK (t-20)    20
MEH | USGC | FCC (t-20)    20
WTI | P2 | FCC (t-20)      20
WTI | P2 | HSK (t-20)      20
Length: 205, dtype: int64


In [29]:
df_lag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704 entries, 0 to 703
Columns: 205 entries, target_position to WTI | P2 | HSK (t-20)
dtypes: float64(180), int64(4), object(21)
memory usage: 1.1+ MB


In [30]:
df_lag = df_lag.dropna()

In [31]:
df_lag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 20 to 703
Columns: 205 entries, target_position to WTI | P2 | HSK (t-20)
dtypes: float64(180), int64(4), object(21)
memory usage: 1.1+ MB


In [32]:
# df_input=df_lag.drop(columns='action',axis=1) # since these columns wer
df_input=df_lag.copy()
# df_output=df_lag['action']

In [33]:
df_cat = df_input.select_dtypes(include=['object']).copy()

In [34]:
df_cat.head()

,target_position,target_position (t-1),target_position (t-2),target_position (t-3),target_position (t-4),target_position (t-5),target_position (t-6),target_position (t-7),target_position (t-8),target_position (t-9),...,target_position (t-11),target_position (t-12),target_position (t-13),target_position (t-14),target_position (t-15),target_position (t-16),target_position (t-17),target_position (t-18),target_position (t-19),target_position (t-20)
20,SHORT,SHORT,SHORT,LONG,LONG,SHORT,SHORT,LONG,LONG,LONG,...,SHORT,SHORT,SHORT,LONG,SHORT,LONG,SHORT,LONG,LONG,LONG
21,SHORT,SHORT,SHORT,SHORT,LONG,LONG,SHORT,SHORT,LONG,LONG,...,SHORT,SHORT,SHORT,SHORT,LONG,SHORT,LONG,SHORT,LONG,LONG
22,LONG,SHORT,SHORT,SHORT,SHORT,LONG,LONG,SHORT,SHORT,LONG,...,LONG,SHORT,SHORT,SHORT,SHORT,LONG,SHORT,LONG,SHORT,LONG
23,LONG,LONG,SHORT,SHORT,SHORT,SHORT,LONG,LONG,SHORT,SHORT,...,LONG,LONG,SHORT,SHORT,SHORT,SHORT,LONG,SHORT,LONG,SHORT
24,SHORT,LONG,LONG,SHORT,SHORT,SHORT,SHORT,LONG,LONG,SHORT,...,LONG,LONG,LONG,SHORT,SHORT,SHORT,SHORT,LONG,SHORT,LONG


In [35]:
#categorical data
categorical_cols = df_cat.columns 


In [36]:
df=MultiColumnLabelEncoder(columns = categorical_cols).fit_transform(df_input)    # 0 means buy, 1 means sell

In [37]:
# df_n=normalize_df(df)
df_n=df.copy()

In [38]:
df_n.head()

,target_position,year,month,month_day,weekday,time_spread (t-1),dod_change (t-1),target_position (t-1),MEH-WTI (t-1),MID (t-1),...,time_spread (t-20),dod_change (t-20),target_position (t-20),MEH-WTI (t-20),MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20)
20,1,2018,1,16,1,0.07,-0.05,1,4.20,1.55,...,-1.07,0.26,0,-0.50,0.55,1.80,0.87,1.76,1.91,0.66
21,1,2018,1,17,2,0.06,-0.01,1,4.05,1.35,...,-1.91,0.20,0,2.00,0.20,2.75,0.54,0.85,1.33,1.85
22,0,2018,1,18,3,0.05,-0.01,1,4.05,1.30,...,-1.61,0.09,0,1.85,-0.50,2.35,-1.33,0.28,2.73,2.37
23,0,2018,1,19,4,0.06,0.01,0,3.80,1.15,...,-1.00,-0.41,1,0.80,-0.60,1.85,1.28,2.00,3.47,3.23
24,1,2018,1,22,0,0.06,0.00,0,3.55,1.00,...,-0.36,0.23,0,1.65,0.00,1.50,1.60,1.74,5.78,5.99


In [39]:
# df_num=df_input[~df_input.isin(df_cat)].dropna(axis=1)
# df=df_input.fillna(0)

In [40]:
# frames = [df_n_input, df_output]

# df_final = pd.concat(frames)

In [41]:
df_n.head()


,target_position,year,month,month_day,weekday,time_spread (t-1),dod_change (t-1),target_position (t-1),MEH-WTI (t-1),MID (t-1),...,time_spread (t-20),dod_change (t-20),target_position (t-20),MEH-WTI (t-20),MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20)
20,1,2018,1,16,1,0.07,-0.05,1,4.20,1.55,...,-1.07,0.26,0,-0.50,0.55,1.80,0.87,1.76,1.91,0.66
21,1,2018,1,17,2,0.06,-0.01,1,4.05,1.35,...,-1.91,0.20,0,2.00,0.20,2.75,0.54,0.85,1.33,1.85
22,0,2018,1,18,3,0.05,-0.01,1,4.05,1.30,...,-1.61,0.09,0,1.85,-0.50,2.35,-1.33,0.28,2.73,2.37
23,0,2018,1,19,4,0.06,0.01,0,3.80,1.15,...,-1.00,-0.41,1,0.80,-0.60,1.85,1.28,2.00,3.47,3.23
24,1,2018,1,22,0,0.06,0.00,0,3.55,1.00,...,-0.36,0.23,0,1.65,0.00,1.50,1.60,1.74,5.78,5.99


In [42]:
df_n.to_csv('features.csv')